In [1]:
import numpy as np
import tensorflow as tf
import os

# load data

In [2]:
train, test_val = tf.keras.datasets.mnist.load_data()

# Data preprocessing

In [3]:
# scale data to obtain data between [0, 1]
user_test, train, test_val = (train[0][:10] / 255.0, train[1][:10]), \
                           (train[0][10:] / 255.0, train[1][10:]), \
                           (test_val[0] / 255.0, test_val[1])

In [4]:
# split test_val into validation, and test
VAL2TEST_RATIO = 0.6
NUM_VAL = int(test_val[0].shape[0] * VAL2TEST_RATIO)
validation = (test_val[0][:NUM_VAL,:,:], test_val[1][:NUM_VAL,])
test = (test_val[0][NUM_VAL:,:,:], test_val[1][NUM_VAL:,])

In [5]:
# shuffle and batch data
BUFFER_SIZE = 10000
BATCH_SIZE = 50

train_ds = tf.data.Dataset.from_tensor_slices(train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_ds = tf.data.Dataset.from_tensor_slices(validation).batch(validation[0].shape[0])
test_ds = tf.data.Dataset.from_tensor_slices(test).batch(test[0].shape[0])
user_ds = tf.data.Dataset.from_tensor_slices(user_test).batch(user_test[0].shape[0])

# Build model

In [6]:
input_shape = (28, 28, 1)
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape, name="flatten"),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax', name='last_layer')
])

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(optimizer='adam', loss=loss_object, metrics=['accuracy'])

In [8]:
NUM_EPOCHS = 10

model.fit(train_ds, epochs=NUM_EPOCHS, validation_data=validation_ds, verbose=2)

Epoch 1/10
1200/1200 - 4s - loss: 0.2646 - accuracy: 0.9236 - val_loss: 0.1695 - val_accuracy: 0.9477 - 4s/epoch - 3ms/step
Epoch 2/10
1200/1200 - 3s - loss: 0.1129 - accuracy: 0.9662 - val_loss: 0.1275 - val_accuracy: 0.9603 - 3s/epoch - 3ms/step
Epoch 3/10
1200/1200 - 3s - loss: 0.0781 - accuracy: 0.9763 - val_loss: 0.1199 - val_accuracy: 0.9597 - 3s/epoch - 3ms/step
Epoch 4/10
1200/1200 - 3s - loss: 0.0614 - accuracy: 0.9800 - val_loss: 0.1184 - val_accuracy: 0.9648 - 3s/epoch - 3ms/step
Epoch 5/10
1200/1200 - 3s - loss: 0.0492 - accuracy: 0.9843 - val_loss: 0.1138 - val_accuracy: 0.9655 - 3s/epoch - 3ms/step
Epoch 6/10
1200/1200 - 3s - loss: 0.0383 - accuracy: 0.9876 - val_loss: 0.1096 - val_accuracy: 0.9663 - 3s/epoch - 3ms/step
Epoch 7/10
1200/1200 - 3s - loss: 0.0316 - accuracy: 0.9896 - val_loss: 0.1243 - val_accuracy: 0.9658 - 3s/epoch - 3ms/step
Epoch 8/10
1200/1200 - 4s - loss: 0.0267 - accuracy: 0.9912 - val_loss: 0.1296 - val_accuracy: 0.9668 - 4s/epoch - 3ms/step
Epoch 9/

In [9]:
save_path = os.path.join("model_2")
tf.saved_model.save(model, save_path)

INFO:tensorflow:Assets written to: model_2\assets


# model evaluate

In [10]:
my_model = tf.saved_model.load("./model_2")

In [11]:
prediction = my_model.signatures["serving_default"]

In [12]:
# expected: TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='flatten_input')
user_input = tf.convert_to_tensor(user_test[0][0], np.float32)
prediction(flatten_input=user_input), user_test[1][0]

({'last_layer': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
  array([[1.0450383e-14, 2.8049621e-08, 3.5840955e-10, 1.7918159e-01,
          1.3730888e-20, 8.2081836e-01, 4.5585274e-18, 3.0990164e-09,
          4.0964397e-13, 4.8129962e-09]], dtype=float32)>},
 5)